In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import warnings
import sys
sys.path.append('../')
pd.options.mode.chained_assignment = None 



In [2]:
files = open("bees/files.txt")
filenames = []
while True:
    line = files.readline().strip()
    if not line:
        break
    filenames.append("bees/" + line)

vdf = pd.concat(map(pd.read_csv, filenames), ignore_index=True)

In [3]:
vdf = vdf.drop_duplicates(subset=['track_starttime','track_tagid'], keep='last')

In [4]:
vdf['track_endtime'] = vdf['track_endtime'].apply(lambda x: pd.to_datetime(x))
vdf['track_starttime'] = vdf['track_starttime'].apply(lambda x: pd.to_datetime(x))
vdf['track_tagid'] = vdf['track_tagid'].apply(lambda x: str(x))

In [5]:
vdf = vdf.sort_values(by='track_starttime')

In [6]:
#SECONDS THRESHOLD
#used to classify multiple detections into part of a single event
#also used to tell when two detections are part of different events
#by checking the time distance between them
t = 15

#DISTANCE THRESHOLD
#used to classify an event as entering or exiting
#when two consecutive detections of the same event 
#have this distance in y position, they are utilized to predict
#the trajectory. If not then it checks the detection prior for
#the distance threshold, and continues doing so until it finds
#the last detection in the event or until it finds a distance
#of more than the threshold

t2 = 300

#ANGLE THRESHOLD
#used to generate angle ranges for classifying as exiting
#or entering

angle = 10


## Look at events prior and classify based on y displacement

In [7]:
def beeCleanPrior(bee):

    id = bee['track_tagid'].iloc[0]

    new_event = []
    datetime = []
        
    for i in range(len(bee)-1):

            time = bee['track_endtime'].iloc[i]
            next_t = bee['track_starttime'].iloc[i+1]

            init = bee['track_starty'].iloc[i]

            #if time passed is greater than the threshold,
            #classify the last detection in an event
            if (next_t - time).total_seconds() > t:
                second = bee['track_endy'].iloc[i]
                matched = False
                counter = 1
                #iterate backwards until the distance threshold
                #is met or until they reach the first detection in
                #the event
                while not matched: 
                    if abs(second-init) > t2:
                        if second < init:
                            new_event.append('exiting')
                        else:
                            new_event.append('entering')
                        matched = True
                    elif i - counter > 0 and (time - bee['track_starttime'].iloc[i-counter]).total_seconds() < t:
                        init = bee['track_starty'].iloc[i-counter]
                        counter += 1
                    else:
                        if second < init:
                            new_event.append('exiting')
                        elif init < second:
                            new_event.append('entering')
                        else:
                            new_event.append('unknown')
                        matched = True
                datetime.append(time)
                
                
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    return df

In [8]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanPrior(b)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0).reset_index() 
new = new[['tagID','datetime','event']]
new.to_csv("bee_prior.csv", index=False)
prior = new
new


C:\Users\kashi\AppData\Local\Temp\ipykernel_1276\3056580212.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0).reset_index()


,tagID,datetime,event
0,2623,2019-07-28 16:01:06.200,entering
1,2623,2019-07-28 16:03:35.650,entering
2,2623,2019-07-28 16:04:21.550,entering
3,2623,2019-07-28 16:05:43.700,exiting
4,2623,2019-07-28 16:06:23.700,exiting
...,...,...,...
18080,2575,2019-08-21 11:52:24.000,exiting
18081,2575,2019-08-21 13:46:59.100,exiting
18082,2575,2019-08-21 13:58:11.100,exiting
18083,2575,2019-08-21 18:41:04.900,exiting


## Classify events based on summed vector angle of all detections corresponding to an event

Obtained from BeeCam-AprilTag
https://github.com/AERS-Lab/BeeCam-AprilTag

In [9]:
def beeCleanAngle(bee):

    id = bee['track_tagid'].iloc[0]
    new_event = []
    datetime = []
    vector_x = []
    vector_y = []

    
    exit_min = 180 + angle
    exit_max = 360 - angle
    enter_min = angle
    enter_max = 180 - angle
        
    
    for i in range(len(bee)-1):

            time = bee['track_endtime'].iloc[i]
            next_t = bee['track_starttime'].iloc[i+1]

            #obtain direction vector 
            #normalized to a magnitude of 1
        
            angle_deg = bee['angle'].iloc[i]
            unit_dx = np.cos(np.deg2rad(angle_deg))
            unit_dy = np.sin(np.deg2rad(angle_deg))

            vector_x.append(unit_dx)
            vector_y.append(unit_dy)
            
            #if next detection is further than the threshold,
            #utilize all stored angles to calculate event trajectory
            if (next_t - time).total_seconds() > t:
                #obtain average
                avg_x = sum(vector_x)/len(vector_x)
                avg_y = sum(vector_y)/len(vector_y)

                if avg_x == 0 and avg_y == 0:
                    deg = 0
                elif avg_x == 0 and avg_y != 0:
                    if avg_y > 0:
                        deg = 270
                    elif avg_y < 0:
                        deg = 90
                else:
                    # determine direction angle using arctan
                    deg = np.rad2deg(np.arctan(avg_y/avg_x))
                    print(deg)
                    
                    # since arctan limits are (-90,90), use coordinate directions to 
                    # correct the angle to be within standard [0,360) range
                    if avg_x > 0 and avg_y >= 0:
                        deg = deg
                    elif avg_x < 0 and avg_y >= 0:
                        deg = 180 + deg
                    elif avg_x < 0 and avg_y < 0:
                        deg = deg + 180
                    elif avg_x > 0 and avg_y < 0:
                        deg = 360 + deg

                #classify events based on angle threshold
                if deg >= exit_min and deg <= exit_max:
                    new_event.append('exiting')
                elif deg >= enter_min and deg <= enter_max:
                    new_event.append('entering')
                else:
                    new_event.append('unknown')
                datetime.append(time)
                vector_x = []
                vector_y = []
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    #print(bee)
    return df

In [10]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanAngle(b)
    #print(events)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0).reset_index() 
new = new[['tagID','datetime','event']]
new.to_csv("bee_angle.csv", index=False)
summed = new
new


79.38682416996403
-77.34828095570825
69.14295354541045
78.53585063400084
86.5546960930019
-69.19568421748787
39.38996485436406
-11.880942862701117
-67.9427943403793
53.30057580959091
57.09088159223262
-88.85924008880785
-5.1517355153197135
51.84474224266067
-56.68731481391871
8.380458766930444
-20.94518426785132
-10.889701357535731
2.9784491562043303
-73.31731110854818
2.53337965091898
-79.86612427681192
3.594849663787745
70.04827844502724
-82.36274269867366
-64.22964708435238
-79.81458534024667
-83.44719939943654
-83.74707211248808
-61.82156600212593
63.96323633809589
61.34104638369709
81.4432849096836
-63.54416020784304
82.54465181775872
-47.11404916523217
-21.613438726288038
-9.062017964902369
32.20280211079801
88.47569025280141
53.55881972554294
78.08945590169962
-51.7653767953008
9.688728272742527
-79.47399937303635
66.30615452428324
33.97593820835673
-8.714300778342901
-78.20480527686573
-4.343789651698072
27.71988912062519
2.5809055681503588
-58.68334089938009
82.62053474806038


C:\Users\kashi\AppData\Local\Temp\ipykernel_1276\94246506.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0).reset_index()


,tagID,datetime,event
0,2623,2019-07-28 16:01:06.200,entering
1,2623,2019-07-28 16:03:35.650,entering
2,2623,2019-07-28 16:04:21.550,entering
3,2623,2019-07-28 16:05:43.700,entering
4,2623,2019-07-28 16:06:23.700,entering
...,...,...,...
18080,2575,2019-08-21 11:52:24.000,entering
18081,2575,2019-08-21 13:46:59.100,entering
18082,2575,2019-08-21 13:58:11.100,entering
18083,2575,2019-08-21 18:41:04.900,entering


## Classify events based on last angle corresponding to an event

Modified from BeeCam-AprilTag
https://github.com/AERS-Lab/BeeCam-AprilTag

In [11]:
def beeCleanSingleAngle(bee):

    id = bee['track_tagid'].iloc[0]
    new_event = []
    datetime = []
    
    exit_min = 180 + angle
    exit_max = 360 - angle
    enter_min = angle
    enter_max = 180 - angle
        
    
    for i in range(len(bee)-1):

            time = bee['track_endtime'].iloc[i]
            next_t = bee['track_starttime'].iloc[i+1]

            #if time is over threshold
            #utilize last movement angle to predict trajectory
            if (next_t - time).total_seconds() > t:

                angle_deg = bee['angle'].iloc[i]
                avg_x = np.cos(np.deg2rad(angle_deg))
                avg_y = np.sin(np.deg2rad(angle_deg))

                if avg_x == 0 and avg_y == 0:
                    deg = 0
                elif avg_x == 0 and avg_y != 0:
                    if avg_y > 0:
                        deg = 270
                    elif avg_y < 0:
                        deg = 90
                else:
                    # determine direction angle using arctan
                    deg = np.rad2deg(np.arctan(avg_y/avg_x))
                    
                    # since arctan limits are (-90,90), use coordinate directions to 
                    # correct the angle to be within standard [0,360) range
                    if avg_x > 0 and avg_y >= 0:
                        deg = deg
                    elif avg_x < 0 and avg_y >= 0:
                        deg = 180 + deg
                    elif avg_x < 0 and avg_y < 0:
                        deg = deg + 180
                    elif avg_x > 0 and avg_y < 0:
                        deg = 360 + deg

                #print(deg)
                if deg >= exit_min and deg <= exit_max:
                    new_event.append('exiting')
                elif deg >= enter_min and deg <= enter_max:
                    new_event.append('entering')
                else:
                    new_event.append('unknown')
                datetime.append(time)
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    #print(bee)
    return df

In [12]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanSingleAngle(b)
    #print(events)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0).reset_index() 
new = new[['tagID','datetime','event']]
new.to_csv("bee_singleangle.csv", index=False)
single = new
new


C:\Users\kashi\AppData\Local\Temp\ipykernel_1276\1948743054.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0).reset_index()


,tagID,datetime,event
0,2623,2019-07-28 16:01:06.200,entering
1,2623,2019-07-28 16:03:35.650,entering
2,2623,2019-07-28 16:04:21.550,entering
3,2623,2019-07-28 16:05:43.700,entering
4,2623,2019-07-28 16:06:23.700,entering
...,...,...,...
18080,2575,2019-08-21 11:52:24.000,entering
18081,2575,2019-08-21 13:46:59.100,entering
18082,2575,2019-08-21 13:58:11.100,entering
18083,2575,2019-08-21 18:41:04.900,entering


In [13]:
correct = {'prior':0,'summedvector':0,'singleangle':0}
count = 0
exiting = ['inside_outside']
entering = ['outside_inside']
for index, row in prior.iterrows():
    datetime = row['datetime']
    tagID = row['tagID']
    track_shape = vdf[(vdf['track_tagid'] == tagID) & (vdf['track_endtime'] == datetime)]['track_shape'].iloc[0]
    if track_shape in exiting:
        event = 'exiting'
    elif track_shape in entering:
        event = 'entering'
    else:
        event = 'unknown'

    if event != 'unknown':
        if row['event'] == event:
            correct['prior'] += 1
        if summed['event'].iloc[index] == event:
            correct['summedvector'] += 1
        if single['event'].iloc[index] == event:
            correct['singleangle'] += 1
        count += 1

results = {k: v/count for k, v in correct.items()}
results


{'prior': 1.0, 'summedvector': 0.8530679933665009, 'singleangle': 0.9}

In [14]:
enter = {'prior':0,'summedvector':0}
exit = {'prior':0,'summedvector':0}

entercount = 0
exitcount = 0
exiting = ['inside_outside']
entering = ['outside_inside']
for index, row in prior.iterrows():
    datetime = row['datetime']
    tagID = row['tagID']
    track_shape = vdf[(vdf['track_tagid'] == tagID) & (vdf['track_endtime'] == datetime)]['track_shape'].iloc[0]
    if track_shape in exiting:
        event = 'exiting'
    elif track_shape in entering:
        event = 'entering'
    else:
        event = 'unknown'

    if event == 'entering':
        if row['event'] == event:
            enter['prior'] += 1
        if summed['event'].iloc[index] == event:
            enter['summedvector'] += 1
        entercount += 1
    if event == 'exiting':
        if row['event'] == event:
            exit['prior'] += 1
        if summed['event'].iloc[index] == event:
            exit['summedvector'] += 1
        exitcount += 1



In [15]:
results = {k: v/entercount for k, v in enter.items()}
results

{'prior': 1.0, 'summedvector': 0.9409602076124568}

In [16]:
results = {k: v/exitcount for k, v in exit.items()}
results

{'prior': 1.0, 'summedvector': 0.7984131253362022}